In [3]:
import pandas as pd 
import os

In [ ]:
folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [4]:
df = pd.read_csv('vehicles_df.csv')
df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [7]:
df.describe()

,Unnamed: 0,Public Vehicle Number,Vehicle Model Year,ZIP Code
count,66408.000000,6.640800e+04,58640.000000,59264.000000
mean,8300.500000,3.854566e+04,2012.669714,60612.519573
std,4792.620661,1.372915e+05,83.620667,104.288240
min,0.000000,0.000000e+00,0.000000,60004.000000
25%,4150.000000,3.633000e+03,2013.000000,60619.000000
50%,8300.500000,6.991500e+03,2016.000000,60631.000000
75%,12451.000000,1.684100e+04,2021.000000,60646.000000
max,16601.000000,6.902206e+06,2025.000000,61111.000000
